In [ ]:
from pyspark.sql import SparkSession

import pandas as pd
import time
timestr = time.strftime("%Y%m%d-%H%M%S")

In [ ]:
url = "jdbc:postgresql://localhost:5432/reddit_comments"

properties = {
    "user": "admin",
    "password": "theLegend30$$",
    "driver": "org.postgresql.Driver"
}

spark = SparkSession\
    .builder\
    .appName("Reddit Comments 'January 6th' Extraction")\
    .config("spark.jars", "/usr/local/postgres/postgresql-42.6.0.jar")\
    .getOrCreate()
    
df = spark.read \
    .format("jdbc") \
    .option("url", url)\
    .option("dbtable", "reddit_jan6_agg") \
    .option("user", properties["user"]) \
    .option("password", properties["password"]) \
    .option("driver", properties["driver"]) \
    .load()
    
# df.printSchema()

In [ ]:
df.createOrReplaceTempView("reddit_j6_aggr")
query = """SELECT subreddit, body, created_on FROM reddit_j6_aggr GROUP BY subreddit, body, created_on;"""

reddit_df = spark.sql(query)

In [ ]:
# Data Export to Pandas
jan6DF = reddit_df.toPandas()
spark.stop()
del df, reddit_df 

In [ ]:
# Data Review
jan6DF.info()
jan6DF.describe() 

# 3 unique subreddits with 1563 unique bodies and 182 unique days activity was created
# Top subreddit being Conservative

In [ ]:
jan6DF.to_csv('./RAW_jan6DF_'+timestr+'.csv', index = False)